#Sequence Classification
Sequence classification is the task of classifying sequences according to a given number of classes.

In [2]:
from transformers import pipeline
classifier=pipeline('sentiment-analysis')
result=classifier('I hate you')[0]
print(f"Label:{result['label']},with score:{result['score']}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


Label:NEGATIVE,with score:0.9991129040718079


In [5]:
result= classifier('Hey Beautiful,you are my private property')[0]
print(f"Label:{result['label']}, with score:{result['score']}")

Label:POSITIVE, with score:0.9998416900634766


In [7]:
# Sequence classification using a model to determine if two sequences are paraphrases of each other
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

# Initialize the model and tokenizer
checkpoint='bert-base-cased-finetuned-mrpc'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

classes=['not paraphrase','is paraphrase']

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# Tokenizer will automatically add any model specific separators [cls] [sep]
paraphrase=tokenizer(sequence_0,sequence_2,return_tensors='pt')
not_paraphrase=tokenizer(sequence_0,sequence_1,return_tensors='pt')

# The sentence pair is passed through the model to get logits, which are rew prediction scores.
paraphrase_classification_logits=model(**paraphrase).logits
not_paraphrase_classification_logits=model(**not_paraphrase).logits

# Softmax function converts logits into probabilities that sum to 1
paraphrase_result=torch.softmax(paraphrase_classification_logits,dim=1).tolist()[0]
not_paraphrese_result=torch.softmax(not_paraphrase_classification_logits,dim=1).tolist()[0]

# The result is printed as percenteage probability

# It should be paraphrase
for i in range(len(classes)):
  print(f"{classes[i]}: {int(round(paraphrase_result[i]*100))}%")
print("*"*100)
# It should be not paraphrase
for i in range(len(classes)):
  print(f"{classes[i]}: {int(round(not_paraphrese_result[i]*100))}%")

not paraphrase: 10%
is paraphrase: 90%
****************************************************************************************************
not paraphrase: 94%
is paraphrase: 6%


#Extractive Question Answering
Extractive Question Answering is the task of extracting an answer from a text given a question.

In [8]:
from transformers import pipeline
# Initialze the question-answering  pipeling
question_ansewer=pipeline('question-answering')

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

result=question_ansewer(question='What is extractive question answering?',context=context)
print(f"Answer: {result['answer']}, score: {round(result['score'],4)}, start: {result['start']}, end: {result['end']}")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


Answer: the task of extracting an answer from a text given a question, score: 0.6177, start: 34, end: 95


In [9]:
result1=question_ansewer(question='What is a good example of a question answering dataset?',context=context)
result1
print(f"Answer: {result1['answer']}, score: {round(result1['score'],4)}, start: {result1['start']}, end: {result['end']}")

Answer: SQuAD dataset, score: 0.5152, start: 147, end: 95


In [10]:
from transformers import AutoTokenizer,AutoModelForQuestionAnswering
import torch

# Initialize the model and tokenizer
model= AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer=AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
  inputs =tokenizer(question,text,add_special_tokens=True,return_tensors='pt')
  # This gives you a simple python list of token ID
  input_ids=inputs['input_ids'].tolist()[0]

  outputs=model(**inputs)
  # These are raw output logits.
  answer_start_score=outputs.start_logits
  answer_end_score=outputs.end_logits

  # Get the most likely begining of the answer with argmax of the score
  answer_start=torch.argmax(answer_start_score)
  # Get the most likely ending og the answer with argmax of the score
  answer_end=torch.argmax(answer_end_score)

  answer=tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

  print(f"Question: {question}")
  print(f"Answer: {answer}")

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32
Question: What does 🤗 Transformers provide?
Answer: general - purpose architecture
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytor


# Masked Language Model
Is the task of masking tokens in a sequence with a masking token,and prompting the model to fill the mask with an appropriate token.This allowe the model to attend both right and left side context

In [13]:
from transformers import pipeline
from pprint import pprint

unmasker=pipeline('fill-mask')

pprint(
    unmasker(
        f"Elon musk is creating a {unmasker.tokenizer.mask_token} that people can live on mars."
    )
)

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'score': 0.17677295207977295,
  'sequence': 'Elon musk is creating a myth that people can live on mars.',
  'token': 17721,
  'token_str': ' myth'},
 {'score': 0.04319721832871437,
  'sequence': 'Elon musk is creating a belief that people can live on mars.',
  'token': 6563,
  'token_str': ' belief'},
 {'score': 0.03510730341076851,
  'sequence': 'Elon musk is creating a misconception that people can live on '
              'mars.',
  'token': 36448,
  'token_str': ' misconception'},
 {'score': 0.031594328582286835,
  'sequence': 'Elon musk is creating a reality that people can live on mars.',
  'token': 2015,
  'token_str': ' reality'},
 {'score': 0.03146098181605339,
  'sequence': 'Elon musk is creating a fantasy that people can live on mars.',
  'token': 8235,
  'token_str': ' fantasy'}]


In [15]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

# This will put [MASK] token in the sentence
sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="tf")
# This will locate the position of [MASK] in the input list
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

# The model outputs scores for every token at ever position
token_logits = model(**inputs).logits
# Predicted scores for all vocabulary tokens at the masked position
mask_token_logits = token_logits[0, mask_token_index, :]

# Finds the top 5 tokens with highest scores
top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

All PyTorch model weights were used when initializing TFDistilBertForMaskedLM.

All the weights of TFDistilBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


# Casual Language Modeling
Is the task of predicting the token following sequence of the token.The model only attends the left side context

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer,tf_top_k_top_p_filtering
import torch
import torch.nn as nn

tokenizer=AutoTokenizer.from_pretrained('gpt2')
model=AutoModelForCausalLM.from_pretrained('gpt2')

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs=tokenizer(sequence,return_tensors='pt')
input_ids=inputs['input_ids']

# Get logits of hidden state
# Extracting logits only for last token
next_token_logits=model(**inputs).logits[:,-1,:]

# Filter
filtered_next_token_logits=generate(next_token_logits,top_k=50,top_p=1.0)

# sample
probs=nn.functional.softmax(filtered_next_token_logits,dim=-1)
next_token=torch.multinomial(probs,num_samples=1)

generated=torch.cat([input_ids,next_token],dim=-1)

resulting_string=tokenizer.decode(generated.tolist()[0])
print(resulting_string)

# Text Generation

In [4]:
from transformers import pipeline
text_generator=pipeline('text-generation')
print(text_generator('Elon musk is a business tycoon',max_length=50,do_sample=False))

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Elon musk is a business tycoon who has been a major player in the world of music since the early 1990s. He has been a major player in the music industry for over 30 years, and has been a major player in the music'}]


In [5]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

model=AutoModelForCausalLM.from_pretrained('xlnet-base-cased')
tokenizer=AutoTokenizer.from_pretrained('xlnet-base-cased')

PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "

inputs=tokenizer(PADDING_TEXT+ prompt , add_special_tokens=False,return_tensors='pt')['input_ids']
print(inputs)
print('*'*100)
prompt_len=len(tokenizer.decode(inputs[0]))
output=model.generate(inputs,max_length=250,do_sample=True,top_p=0.50,top_k=60)
generated=prompt + tokenizer.decode(output[0])[prompt_len +1:]
print(generated)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

tensor([[   67,  2840,    19,    18,  1484,    20,   965, 29077,  8719,  1273,
            21,    45,   273,    17,    10, 15048,    28, 27511,    21,  4185,
            11,    41,  2444,     9,    32,  1025,    20,  8719,    26,    23,
           673,   966,    19, 29077, 20643, 27511, 20822, 20643,    19,    17,
          6616, 17511,    18,  8978,    20,    18,   777,     9, 19233,  1527,
         17669,    19,    24,   673,    17, 28756,   150, 12943,  4354,   153,
            27,   442,    37,    45,   668,    21,    24,   256,    20,   416,
            22,  2771,  4901,     9, 12943,  4354,   153,    51,    24,  3004,
            21, 28142,    23,    65,    20,    18,   416,    34,    24,  2958,
         22947,     9,  1177,    45,   668,  3097, 13768,    23,   103,    28,
           441,   148,    48, 20522,    19, 12943,  4354,   153, 12860,    34,
            18,   326,    27, 17492,   684,    21,  6709,     9,  8585,   123,
           266,    19, 12943,  4354,   153,  6872,  

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (-1). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Today the weather is really nice and I am planning on going to a "no fries" party for my sister and her friend. It will be a really fun day, but it will be a very nice one.<eop><eod> I am doing a lot of research. I think that the first thing I will do is to look at the web site for a new website. I am looking at the website for a


Named Entity Recognition (NER) is the task of classifying tokens according to a class, for example, identifying a token
as a person, an organisation or a location. An example of a named entity recognition dataset is the CoNLL-2003 dataset,
which is entirely based on that task. If you would like to fine-tune a model on an NER task

Here is an example of using pipelines to do named entity recognition, specifically, trying to identify tokens as
belonging to one of 9 classes:

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location


In [6]:
from transformers import pipeline

ner_sq=pipeline('ner')

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

for ner in ner_sq(sequence):
  print(ner)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cpu


{'entity': 'I-ORG', 'score': np.float32(0.99957865), 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': np.float32(0.9909764), 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': np.float32(0.9982224), 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-ORG', 'score': np.float32(0.9994879), 'index': 4, 'word': 'Inc', 'start': 13, 'end': 16}
{'entity': 'I-LOC', 'score': np.float32(0.9994344), 'index': 11, 'word': 'New', 'start': 40, 'end': 43}
{'entity': 'I-LOC', 'score': np.float32(0.99931955), 'index': 12, 'word': 'York', 'start': 44, 'end': 48}
{'entity': 'I-LOC', 'score': np.float32(0.9993794), 'index': 13, 'word': 'City', 'start': 49, 'end': 53}
{'entity': 'I-LOC', 'score': np.float32(0.98625815), 'index': 19, 'word': 'D', 'start': 79, 'end': 80}
{'entity': 'I-LOC', 'score': np.float32(0.95142686), 'index': 20, 'word': '##UM', 'start': 80, 'end': 82}
{'entity': 'I-LOC', 'score': np.float32(0.9336589), 'index': 21,

In [12]:
from transformers import AutoModelForTokenClassification,AutoTokenizer
import torch

model=AutoModelForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

sequence = (
  "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
  "therefore very close to the Manhattan Bridge."
)

input=tokenizer(sequence,return_tensors='pt')
tokens=input.tokens()

outputs=model(**input).logits
predictions=torch.argmax(outputs,dim=2)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('Hu', 'I-ORG')
('##gging', 'I-ORG')
('Face', 'I-ORG')
('Inc', 'I-ORG')
('.', 'O')
('is', 'O')
('a', 'O')
('company', 'O')
('based', 'O')
('in', 'O')
('New', 'I-LOC')
('York', 'I-LOC')
('City', 'I-LOC')
('.', 'O')
('Its', 'O')
('headquarters', 'O')
('are', 'O')
('in', 'O')
('D', 'I-LOC')
('##UM', 'I-LOC')
('##BO', 'I-LOC')
(',', 'O')
('therefore', 'O')
('very', 'O')
('close', 'O')
('to', 'O')
('the', 'O')
('Manhattan', 'I-LOC')
('Bridge', 'I-LOC')
('.', 'O')
('[SEP]', 'O')


#Summarization
Summarization is the task of summarizing a document or an article into a shorter text.

In [17]:
from transformers import pipeline
summarize=pipeline('summarization')

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""


print(summarize(ARTICLE,max_length=130,min_length=30,do_sample=False))

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . At one time, she was married to eight men at once, prosecutors say .'}]


In [19]:
from transformers import AutoModelForSeq2SeqLM ,AutoTokenizer

# Text To Text Transfer Transformer
model=AutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer=AutoTokenizer.from_pretrained('t5-base')

inputs=tokenizer('summarize'+ ARTICLE , return_tensors='pt',max_length=512,truncation=True)
outputs=model.generate(inputs['input_ids'],max_length=150,min_length=40,length_penalty=2.0,num_beams=4,early_stopping=True)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Liana Barrientos, 39, pleaded not guilty to two counts of "offering a false instrument for filing in the first degree" in 2010 . in total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . prosecutors say the marriages were part of an immigration scam .


 # Translation-
 It allows you to translate text from one language to another

In [22]:
from transformers import pipeline
translate=pipeline('translation_en_to_de')
print(translate('Hey how are you ? hope you are fine !',max_length=40))

No model was supplied, defaulted to google-t5/t5-base and revision a9723ea (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'translation_text': 'Hallo, wie sind Sie ?'}]


In [24]:
from transformers import TFAutoModelForSeq2SeqLM,AutoTokenizer

model=TFAutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer=AutoTokenizer.from_pretrained('t5-base')

inputs=tokenizer("translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt")

output=model.generate(inputs['input_ids'],max_length=40,num_beams=40,early_stopping=True)

print(tokenizer.decode(output[0],skip_special_tokens=True))

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.


#Audio classification
Audio classification assigns a class to an audio signal.

In [27]:
from transformers import pipeline
from datasets import load_dataset
import torch

torch.manual_seed(0)
dataset=load_dataset('hf-internal-testing/librispeech_asr_demo','clean',split='validation')
dataset=dataset.sort('id')
audio_file=dataset[0]['audio']['path']

audio_classifier=pipeline(task='audio-classification',model='ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition')

predictions=audio_classifier(audio_file)
predictions=[{'Score':round(pred['score'],4),'Label':pred['label']}for pred in predictions]
predictions

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition and are newly initialized: ['classifier.bias', 'classifier.weight', '

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Device set to use cpu


[{'Score': 0.1292, 'Label': 'sad'},
 {'Score': 0.129, 'Label': 'neutral'},
 {'Score': 0.1261, 'Label': 'disgust'},
 {'Score': 0.1254, 'Label': 'angry'},
 {'Score': 0.1249, 'Label': 'happy'},
 {'Score': 0.1237, 'Label': 'calm'},
 {'Score': 0.1235, 'Label': 'fearful'},
 {'Score': 0.1182, 'Label': 'surprised'}]

In [28]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

feature_extractor = AutoFeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-ks")
model = AutoModelForAudioClassification.from_pretrained("superb/wav2vec2-base-superb-ks")

inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.argmax(logits, dim=-1).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

'_unknown_'

#Automatic speech recognition
Automatic speech recognition transcribes an audio signal to text.

In [29]:
from transformers import pipeline
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
audio_file = dataset[0]["audio"]["path"]

speech_recognizer = pipeline(task="automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
speech_recognizer(audio_file)

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Device set to use cpu


{'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'}

In [30]:
from transformers import AutoProcessor, AutoModelForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")

inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)
transcription[0]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

#Image classification
Like text and audio classification, image classification assigns a class to an image.

In [31]:
from transformers import pipeline

image_classify=pipeline(task='image-classification')

result=image_classify(images='https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg')
print("\n".join([f"class{d['label']} with score {round(d['score'],4)}"for d in result]))

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 3f49326 (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cpu


classlynx, catamount with score 0.4335
classcougar, puma, catamount, mountain lion, painter, panther, Felis concolor with score 0.0348
classsnow leopard, ounce, Panthera uncia with score 0.0324
classEgyptian cat with score 0.0239
classtiger cat with score 0.0229


In [32]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

feature_extractor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

cats-image.py:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

The repository for huggingface/cats-image contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/huggingface/cats-image.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split: 0 examples [00:00, ? examples/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Egyptian cat
